In [21]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import math

In [2]:
## global variables

VERSION = 1.0
NUM_TILES = 400 # number of tiles to generate per image
TILE_SIZE = 512

In [56]:
## path to directories

raw_data_path = r'\\babyserverdw5\Digital pathology image lib\JHU\Ie-Ming Shih\lymphocytes\230110 dataset\raw'

save_path = r'\\babyserverdw5\Digital pathology image lib\JHU\Ie-Ming Shih\lymphocytes\230110 dataset'

tile_folder_name = '{}x{}_v{}'.format(TILE_SIZE, TILE_SIZE, VERSION)

tile_save_path = os.path.join(save_path, tile_folder_name)

mode = ['/train', '/val', '/test']

if not os.path.exists(tile_save_path):
    os.mkdir(tile_save_path)
    for m in mode:
        os.mkdir(tile_save_path + m)
        os.mkdir(tile_save_path + m + '/images')
        os.mkdir(tile_save_path + m + '/labels')

In [13]:
def normal_crop(rand_cent, padding, img, label):
    # padding is tile_size / 2
    lower_bound = np.asarray(rand_cent) - padding
    upper_bound = np.asarray(rand_cent) + padding
    img_tile = img[int(lower_bound[0]):int(upper_bound[0]), int(lower_bound[1]):int(upper_bound[1]), :]
    label_tile = label[int(lower_bound[0]):int(upper_bound[0]), int(lower_bound[1]):int(upper_bound[1])]
    return img_tile, label_tile

def perform_crop(rand_cent, padding, img, label): 
    img_tile, label_tile = normal_crop(rand_cent, padding, img, label)
    rand_fliplr = np.random.uniform()
    if rand_fliplr >= 0.50: # horizontal flip
        img_tile, label_tile = np.fliplr(img_tile), np.fliplr(label_tile)
    
    rand_rot90 = np.random.randint(1,4) # 90 degree rotation
    img_tile, label_tile = np.rot90(img_tile, k=rand_rot90), np.rot90(label_tile, k=rand_rot90)
    
    return img_tile, label_tile

In [58]:
def create_tiles(im, lb, im_name, m):
    img = cv2.imread(im)
    label = cv2.imread(lb, cv2.IMREAD_UNCHANGED).astype('uint8')
    # get center pixel min/max
    padding = TILE_SIZE / 2 # both indices start at tile_size / 2
    h_max = img.shape[0] - padding
    v_max = img.shape[1] - padding
    tile_counter = 0
    for n in range(NUM_TILES):
        rand_cent = (np.random.randint(padding, h_max+1), np.random.randint(padding, v_max+1))
        img_tile, label_tile = perform_crop(rand_cent, padding, img, label) # crop image based on given center pixel
        fn = im_name + '_0{}'.format(tile_counter)
        tile_counter += 1
        cv2.imwrite(tile_save_path + m + '/images/' + fn + '.tif', img_tile)
        cv2.imwrite(tile_save_path + m + '/labels/'+ fn + '.tif', label_tile)

In [51]:
## create train/val/test split
np.random.seed(0)
images_path = os.path.join(raw_data_path, 'images')
labels_path = os.path.join(raw_data_path, 'labels') 
img_list = np.array(os.listdir(images_path))
tot_imgs = len(img_list)
rand_inds = np.random.permutation(tot_imgs)
train_pt, val_pt, test_pt = math.ceil(0.6 * tot_imgs), math.ceil(0.2 * tot_imgs), None
train_idx, val_idx, test_idx = rand_inds[:train_pt], rand_inds[train_pt:train_pt+val_pt], rand_inds[train_pt+val_pt:]
train_im, val_im, test_im = img_list[train_idx], img_list[val_idx], img_list[test_idx] 
print(train_idx, val_idx, test_idx)

[ 1  6  8  9 14  4  2 13 10] [ 7 11  3] [ 0  5 12]


In [61]:
for m in mode: # for each image
    if m == '/train':
        im_list = train_im
    elif m == '/val':
        im_list = val_im
    else:
        continue
    for fn in im_list:
        im_name = fn[:-4]
        im = os.path.join(images_path, im_name + '.tif')
        lb = os.path.join(labels_path, im_name + '.tif')
        print('processing {}...'.format(im_name))
        create_tiles(im, lb, im_name, m) 

/test
processing 303_roi01_ROI-STIC...


KeyboardInterrupt: 

In [62]:
print(train_im, val_im, test_im)

['303_roi02_ROI-normal.tif' 'FTE363_roi01_ROI-STIC.tif'
 'FTE363_roi03_ROI-normal.tif' 'FTE372 5G_roi01_ROI-STIC.tif'
 'FTE379_roi03_ROI-carcinoma.tif' '305_roi01_ROI-STIC.tif'
 '304_roi01_ROI-STIC.tif' 'FTE379_roi02_ROI-normal.tif'
 'FTE372 5G_roi02_ROI-carcinoma.tif'] ['FTE363_roi02_ROI-carcinoma.tif' 'FTE372 5G_roi03_ROI-normal.tif'
 '304_roi02_ROI-normal.tif'] ['303_roi01_ROI-STIC.tif' '305_roi02_ROI-normal.tif'
 'FTE379_roi01_ROI-STIC.tif']
